Biblioteca Pipeline do PySpark.ml oferece uma forma eficiente e escalável de organizar e executar uma sequência de etapas de transformação de dados e ajuste de modelo em um ambiente distribuído

In [ ]:
import pyspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Nome do aplicativo").getOrCreate()


In [ ]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
carros_temp = spark.read.csv("/home/cliente/download/Carros.csv", inferSchema=True, header=True, sep=";")


In [ ]:
carros = carros_temp.select("consumo", "cilindros", "cilindradas", "hp")

In [ ]:
veccaracteristicas = VectorAssembler(inputCols=[("consumo"), ("cilindros"), ("cilindradas")], outputCol="caracteristicas")

In [ ]:
carros_treino = veccaracteristicas.transform(carros)

In [ ]:
carros_treino.show(3)

+-------+---------+-----------+---+-----------------+
|consumo|cilindros|cilindradas| hp|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
+-------+---------+-----------+---+-----------------+
only showing top 3 rows



In [ ]:
reglin = LinearRegression(featuresCol="caracteristicas",labelCol="hp")

In [ ]:
modelo = reglin.fit(carros_treino)

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
pipeline = Pipeline(stages=[veccaracteristicas,reglin])

In [ ]:
pipelineModel = pipeline.fit(carros)

In [ ]:
previsao = pipelineModel.transform(carros)

In [ ]:
previsao.show()

+-------+---------+-----------+---+------------------+------------------+
|consumo|cilindros|cilindradas| hp|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     21|        6|        160|110|  [21.0,6.0,160.0]|162.32154816816646|
|     21|        6|        160|110|  [21.0,6.0,160.0]|162.32154816816646|
|    228|        4|        108| 93| [228.0,4.0,108.0]| 82.51715587712931|
|    214|        6|        258|110| [214.0,6.0,258.0]|141.86680518718754|
|    187|        8|        360|175| [187.0,8.0,360.0]|202.93528239714834|
|    181|        6|        225|105| [181.0,6.0,225.0]| 145.4980634611832|
|    143|        8|        360|245| [143.0,8.0,360.0]|   207.41448530972|
|    244|        4|       1467| 62|[244.0,4.0,1467.0]| 69.69282676584851|
|    228|        4|       1408| 95|[228.0,4.0,1408.0]| 71.80767356085781|
|    192|        6|       1676|123|[192.0,6.0,1676.0]|132.42483285541724|
|    178|        6|       1676|123|[17